In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

In [3]:
from azure.ai.ml import MLClient
try:
    credential=DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential=InteractivebrowserCredential()

In [4]:
ml_client=MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [7]:
#!pip install mlflow

In [14]:
import pandas as pd 
df=pd.read_csv('diabetes.csv')

In [15]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [16]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [17]:
X,y=df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']], df[['Outcome']]

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train,y_test=train_test_split(X,y, test_size=0.30, random_state=0)

In [22]:
import mlflow
experiment_name='diabetes-mlflow'
mlflow.set_experiment(experiment_name)

2024/03/20 19:52:24 INFO mlflow.tracking.fluent: Experiment with name 'diabetes-mlflow' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1710964344735, experiment_id='de226461-daf6-4265-8f18-479ca333bc4d', last_update_time=None, lifecycle_stage='active', name='diabetes-mlflow', tags={}>

In [25]:
from sklearn.linear_model import LogisticRegression
with mlflow.start_run():
    mlflow.sklearn.autolog()
    LR=LogisticRegression()
    model=LR.fit(x_train, y_train)


2024/03/20 19:57:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/data/pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column

In [27]:
mlflow.sklearn.autolog(disable=True)

In [36]:
y_test.shape

(231, 1)

In [38]:
np.squeeze(y_test).shape

(231,)

In [33]:
model.predict(x_test).shape

(231,)

In [39]:
cm

array([[141,  35],
       [ 16,  39]])

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np 
with mlflow.start_run():
    LR=LogisticRegression()
    model=LR.fit(x_train, y_train)
    cm=confusion_matrix(model.predict(x_test),np.squeeze(y_test))
    acc=accuracy_score(model.predict(x_test),np.squeeze(y_test))
    mlflow.log_metric('accuracy_score_logged',acc)
    mlflow.log

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
